## Compare gradient_descent_stochastic with gradient_descent (batch)

In [17]:
import numpy as np
import matplotlib.pyplot as plt

In [18]:
m = 100000
x = np.random.normal(size=m)
X = x.reshape(-1, 1)
y = 4. * x + 3. + np.random.normal(0, 3, size=m)

In [19]:
def J(theta, X_b, y):
    try:
        return np.sum((y - X_b.dot(theta)) ** 2) / len(y)
    except:
        return float('inf')
    
def dJ(theta, X_b, y):
    return X_b.T.dot(X_b.dot(theta) - y) * 2. / len(y)

def gradient_descent(X_b, y, initial_theta, eta, n_iters=1e4, epsilon=1e-8):
    theta = initial_theta
    cur_iter = 0
    while cur_iter < n_iters:
        gradient = dJ(theta, X_b, y)
        last_theta = theta
        theta = theta - eta * gradient
        if(abs(J(theta, X_b, y) - J(last_theta, X_b, y)) < epsilon):
            break
        cur_iter += 1
    return theta

In [20]:
%%time
X_b = np.hstack((X, np.ones((len(X), 1))))
initial_theta = np.zeros(X_b.shape[1])
eta = 0.01
theta = gradient_descent(X_b, y, initial_theta, eta)
print(theta)

[4.01373637 3.01778093]
CPU times: user 2.49 s, sys: 15.7 ms, total: 2.5 s
Wall time: 422 ms


In [21]:
def dJ_stochastic(theta, X_b_i, y_i):
    return X_b_i.T.dot(X_b_i.dot(theta) - y_i) * 2.

def gradient_descent_stochastic(X_b, y, initial_theta, n_iters=1e4):
    t0 = 5
    t1 = 50
    
    def learning_rate(t):
        return t0 / (t + t1)
    
    theta = initial_theta
    for i in range(n_iters):
        rand_i = np.random.randint(len(X_b))
        gradient = dJ_stochastic(theta, X_b[rand_i], y[rand_i])
        theta = theta - gradient * learning_rate(i)
    return theta

In [22]:
%%time
X_b = np.hstack((X, np.ones((len(X), 1))))
initial_theta = np.zeros(X_b.shape[1])
eta = 0.01
theta = gradient_descent_stochastic(X_b, y, initial_theta, n_iters=len(X_b)//3)
print(theta)

[4.01456586 3.05659441]
CPU times: user 1.15 s, sys: 8.36 ms, total: 1.16 s
Wall time: 241 ms


## 

In [23]:
from fun_machine_learning.linear_regression import LinearRegression 

In [24]:
lin_reg = LinearRegression()
lin_reg.fit_sgd(X, y, n_iters = 2)
lin_reg.coef_, lin_reg.interception_

(array([4.01073049]), 3.0022734155925135)

In [25]:
from sklearn import datasets
boston = datasets.load_boston()
X = boston.data
y = boston.target
X = X[y < 50.0]
y = y[y < 50.0]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=600)

In [26]:
from sklearn.preprocessing import StandardScaler
standardScaler = StandardScaler()
standardScaler.fit(X_train)
X_train_standard = standardScaler.transform(X_train)
X_test_standard = standardScaler.transform(X_test)

In [27]:
from fun_machine_learning.linear_regression import LinearRegression 
lin_reg2 = LinearRegression()
%time lin_reg2.fit_sgd(X_train_standard, y_train, n_iters=2)
lin_reg2.score(X_test_standard, y_test)

CPU times: user 5.38 ms, sys: 2.85 ms, total: 8.23 ms
Wall time: 5.67 ms


0.5997931086671258

In [28]:
%time lin_reg2.fit_sgd(X_train_standard, y_train, n_iters=100)
lin_reg2.score(X_test_standard, y_test)

CPU times: user 162 ms, sys: 6 ms, total: 168 ms
Wall time: 164 ms


0.7350928974423687

## Stochastic gradient descent in sklearn

In [29]:
from sklearn.linear_model import SGDRegressor
sgd_reg = SGDRegressor()
%time sgd_reg.fit(X_train_standard, y_train)
sgd_reg.score(X_test_standard, y_test)

CPU times: user 1.49 ms, sys: 693 µs, total: 2.19 ms
Wall time: 1.25 ms


0.7296444980809615

In [30]:
sgd_reg = SGDRegressor(n_iter_no_change=100)
%time sgd_reg.fit(X_train_standard, y_train)
sgd_reg.score(X_test_standard, y_test)

CPU times: user 10.7 ms, sys: 842 µs, total: 11.5 ms
Wall time: 10.2 ms


0.7348020929623629

## Adjusing gradient

In [36]:
np.random.seed(600)
X = np.random.random(size=(1000, 10))
X_b = np.hstack((X, np.ones((len(X), 1))))
true_theta = np.arange(1, 12, dtype=float)
y = X_b.dot(true_theta) + np.random.normal(size=1000)
X.shape, y.shape

((1000, 10), (1000,))

## Although slow, dJ_debug is a good way to test the results of many similar functions

In [50]:
def dJ_debug(theta, X_b, y, epsilon=0.01):
    res = np.empty(len(theta))
    for i in range(len(theta)):
        theta_1 = theta.copy()
        theta_1[i] += epsilon
        theta_2 = theta.copy()
        theta_2[i] -= epsilon
        res[i] = (J(theta_1, X_b, y) - J(theta_2, X_b, y)) / (2 * epsilon)
    return res

def J(theta, X_b, y):
    try:
        return np.sum((y - X_b.dot(theta)) ** 2) / len(y)
    except:
        return float('inf')
    
def dJ_math(theta, X_b, y):
    return X_b.T.dot(X_b.dot(theta) - y) * 2. / len(y)

def gradient_descent(dJ, X_b, y, initial_theta, eta, n_iters = 1e4, epsilon=1e-8):
    theta = initial_theta
    cur_iter = 0
    while cur_iter < n_iters:
        gradient = dJ(theta, X_b, y)
        last_theta = theta
        theta = theta - eta * gradient
        if(abs(J(theta, X_b, y) - J(last_theta, X_b, y)) < epsilon):
            break
        cur_iter += 1
    return theta

In [51]:
initial_theta = np.zeros(X_b.shape[1])
eta = 0.01
%time theta = gradient_descent(dJ_debug, X_b, y, initial_theta, eta)
theta

CPU times: user 1.57 s, sys: 4.76 ms, total: 1.57 s
Wall time: 1.57 s


array([ 1.25251777,  2.07005404,  3.10992033,  4.26874008,  4.92296325,
        6.08289583,  7.08476308,  7.77844027,  8.76084821,  9.99470387,
       10.77658517])

In [49]:
%time theta = gradient_descent(dJ_math, X_b, y, initial_theta, eta)
theta

CPU times: user 1.38 s, sys: 8.62 ms, total: 1.39 s
Wall time: 232 ms


array([ 1.25251777,  2.07005404,  3.10992033,  4.26874008,  4.92296325,
        6.08289583,  7.08476308,  7.77844027,  8.76084821,  9.99470387,
       10.77658517])